## Import Library

In [1]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
import keras
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
import urllib.request
import zipfile
import os
from keras.models import Sequential
from keras.layers import Embedding,Bidirectional,LSTM,GRU,Dense
import nltk
from nltk.tokenize import word_tokenize
import warnings
nltk.download('punkt')
warnings.filterwarnings('ignore')


# All zipped files must be unzipped and left within the submission folder to run the code.
# Might give errors if ran in windows. Code processed and ran using MAC

/Users/Joven/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/Users/Joven/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and g

## Load dataset

In [2]:
f=open('train.txt','r')
x_train=[]
y_train=[]
for i in f:
    l=i.split(';')
    y_train.append(l[1].strip())
    x_train.append(l[0])
f=open('test.txt','r')
x_test=[]
y_test=[]
for i in f:
    l=i.split(';')
    y_test.append(l[1].strip())
    x_test.append(l[0])
f=open('val.txt','r')
for i in f:
    l=i.split(';')
    y_test.append(l[1].strip())
    x_test.append(l[0])
data_train=pd.DataFrame({'Text':x_train,'Emotion':y_train})
data_test=pd.DataFrame({'Text':x_test,'Emotion':y_test})
data=data_train.append(data_test,ignore_index=True)

## Clean data

Remove all the prepositions, articles, punctuation marks, stop words, leaving only the important words in the sentences

In [3]:
def clean_text(data):
    data=re.sub(r"(#[\d\w\.]+)", '', data)
    data=re.sub(r"(@[\d\w\.]+)", '', data)
    data=word_tokenize(data)
    return data
texts=[' '.join(clean_text(text)) for text in data.Text]
texts_train=[' '.join(clean_text(text)) for text in x_train]
texts_test=[' '.join(clean_text(text)) for text in x_test]

## Tokenization

 It tokenizes each sentence, extracts each unique word and creates a dictionary where each unique word is assigned an index.

In [4]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(texts)
sequence_train=tokenizer.texts_to_sequences(texts_train)
sequence_test=tokenizer.texts_to_sequences(texts_test)
index_of_words=tokenizer.word_index
vocab_size=len(index_of_words)+1

The data-set which we obtained has six unique results or emotions, namely: anger, sadness, fear, joy, surprise and love.

In [5]:
num_classes=6
embed_num_dims=300
max_seq_len=500
class_names=['anger','sadness','fear','joy','surprise','love']
X_train_pad=pad_sequences(sequence_train,maxlen=max_seq_len)
X_test_pad=pad_sequences(sequence_test,maxlen=max_seq_len)
encoding={'anger':0,'sadness':1,'fear':2,'joy':3,'surprise':4,'love':5}
y_train=[encoding[x] for x in data_train.Emotion]
y_test=[encoding[x] for x in data_test.Emotion]
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

In [6]:
def create_embedding_matrix(filepath,word_index,embedding_dim):
    vocab_size=len(word_index)+1
    embedding_matrix=np.zeros((vocab_size,embedding_dim))
    with open(filepath) as f:
        for line in f:
            word,*vector=line.split()
            if word in word_index:
                idx=word_index[word]
                embedding_matrix[idx] = np.array(vector,dtype=np.float32)[:embedding_dim]
    return embedding_matrix
fname='wiki-news-300d-1M.vec'
embedd_matrix=create_embedding_matrix(fname,index_of_words,embed_num_dims)

## Train Model

Embedding layer ----------> Bi-directional layer -----------> Dense layer

In [7]:
embedd_layer=Embedding(vocab_size,embed_num_dims,input_length=max_seq_len,weights=[embedd_matrix],trainable=False)
gru_output_size=128
bidirectional=True
model=Sequential()
model.add(embedd_layer)
model.add(Bidirectional(GRU(units=gru_output_size,dropout=0.2,recurrent_dropout=0.2)))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 300)          5128500   
                                                                 
 bidirectional (Bidirectiona  (None, 256)              330240    
 l)                                                              
                                                                 
 dense (Dense)               (None, 6)                 1542      
                                                                 
Total params: 5,460,282
Trainable params: 331,782
Non-trainable params: 5,128,500
_________________________________________________________________


In [9]:
batch_size=128
epochs=8
hist=model.fit(X_train_pad,y_train,batch_size=batch_size,epochs=epochs,validation_data=(X_test_pad,y_test))

Epoch 1/8
125/125 [==============================] - 492s 4s/step - loss: 1.3955 - accuracy: 0.4773 - val_loss: 1.1177 - val_accuracy: 0.5943
Epoch 2/8
125/125 [==============================] - 14079s 114s/step - loss: 0.9115 - accuracy: 0.6873 - val_loss: 0.6476 - val_accuracy: 0.7790
Epoch 3/8
125/125 [==============================] - 13451s 108s/step - loss: 0.5299 - accuracy: 0.8133 - val_loss: 0.3742 - val_accuracy: 0.8720
Epoch 4/8
125/125 [==============================] - 506s 4s/step - loss: 0.3389 - accuracy: 0.8801 - val_loss: 0.2576 - val_accuracy: 0.9100
Epoch 5/8
125/125 [==============================] - 674s 5s/step - loss: 0.2617 - accuracy: 0.9064 - val_loss: 0.2104 - val_accuracy: 0.9165
Epoch 6/8
125/125 [==============================] - 2190s 18s/step - loss: 0.2134 - accuracy: 0.9183 - val_loss: 0.1843 - val_accuracy: 0.9260
Epoch 7/8
125/125 [==============================] - 566s 5s/step - loss: 0.1942 - accuracy: 0.9209 - val_loss: 0.1715 - val_accuracy: 0.9

 Map the categorical value back to it’s actual English emotion

In [10]:
def dialogue():
    a = [input('Describe how your day is going:')]     # Question 1
    seq=tokenizer.texts_to_sequences(a)
    padded=pad_sequences(seq,maxlen=max_seq_len)
    pred=model.predict(padded)
    print('Message:'+str(a))
    print('Emotion:',class_names[np.argmax(pred)])
    b = [input('What are the thoughts that are going through your mind?')]     # Question 2
    seq=tokenizer.texts_to_sequences(b)
    padded=pad_sequences(seq,maxlen=max_seq_len)
    pred=model.predict(padded)
    print('Message:'+str(b))
    print('Emotion:',class_names[np.argmax(pred)])
    c = [input('Describe your emotions using only action words:')]       # Question 3
    seq=tokenizer.texts_to_sequences(c)
    padded=pad_sequences(seq,maxlen=max_seq_len)
    pred=model.predict(padded)
    print('Message:'+str(c))
    print('Emotion:',class_names[np.argmax(pred)])
    return

In [14]:
dialogue()

Describe how your day is going: My mum fell sick with covid and I am very worried


Message:['My mum fell sick with covid and I am very worried']
Emotion: fear


What are the thoughts that are going through your mind? I am feeling miserable as I can't physically see my mum due to the isolation


Message:["I am feeling miserable as I can't physically see my mum due to the isolation"]
Emotion: sadness


Describe your emotions using only action words: hyperventilating and shaking


Message:['hyperventilating and shaking']
Emotion: fear
